In [7]:
from datetime import datetime, date

In [3]:
now = datetime.now()

In [11]:
date.today().strftime("%Y-%m-%d")

'2022-11-03'

In [1]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF

In [3]:
g = Graph()
g.bind("foaf", FOAF)

bob = URIRef("http://example.org/people/Bob")
linda = BNode()  # a GUID is generated

name = Literal("Bob")
age = Literal(24)

g.add((bob, RDF.type, FOAF.Person))
g.add((bob, FOAF.name, name))
g.add((bob, FOAF.age, age))
g.add((bob, FOAF.knows, linda))
g.add((linda, RDF.type, FOAF.Person))
g.add((linda, FOAF.name, Literal("Linda")))

print(g.serialize(format='xml'))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:foaf="http://xmlns.com/foaf/0.1/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:nodeID="N775c0f14004741e98b0919b1563f4c49">
    <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
    <foaf:name>Linda</foaf:name>
  </rdf:Description>
  <rdf:Description rdf:about="http://example.org/people/Bob">
    <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
    <foaf:name>Bob</foaf:name>
    <foaf:age rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">24</foaf:age>
    <foaf:knows rdf:nodeID="N775c0f14004741e98b0919b1563f4c49"/>
  </rdf:Description>
</rdf:RDF>



In [5]:
uri = 'http://id.loc.gov/resources/works/1893399.bibframe.rdf'
g = Graph()
g.parse(uri)

<Graph identifier=N3fbb92ab2cfe46808511e64b33f28728 (<class 'rdflib.graph.Graph'>)>

In [6]:
g.serialize('conjecturas.nt')

<Graph identifier=N3fbb92ab2cfe46808511e64b33f28728 (<class 'rdflib.graph.Graph'>)>

In [1]:
ass'Science--Methodology'.split("--")

['Science', 'Methodology']